https://github.com/moorissa/medium/blob/master/items-recommender/notebooks/recommendation-MT.ipynb

In [0]:
!pip install turicreate

     |████████████████████████████████| 91.8MB 64kB/s 
     |████████████████████████████████| 86.3MB 50kB/s 
     |████████████████████████████████| 327kB 48.1MB/s 
     |████████████████████████████████| 3.4MB 45.9MB/s 
     |████████████████████████████████| 3.8MB 47.4MB/s 
     |████████████████████████████████| 450kB 44.2MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320850 sha256=2d889e62706437bacadfbd31e120be6cb7e1305259c1b49f624b8658fa5422ed
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=da450d0a488afee2cc4c17cfa641634b026f0be3dcd26d4003a6195422e8067a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built resampy gast
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tens

In [0]:
import graphviz

In [0]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

In [0]:
customers = pd.read_csv('https://raw.githubusercontent.com/moorissa/medium/master/items-recommender/data/recommend_1.csv') 


In [0]:
transactions = pd.read_csv('https://raw.githubusercontent.com/moorissa/medium/master/items-recommender/data/trx_data.csv')


 Data preparation

In [0]:
transactions

,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2
...,...,...
62478,4068,103|103|8|48|126
62479,10918,124|37|37|78|124|124|37|8|35|8
62480,6119,24
62481,8171,167


In [0]:
transactions['products']=transactions['products'].apply(lambda x:[int(i) for i in x.split('|')])

In [0]:
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [0]:
customers.shape, transactions.shape

((1000, 1), (62483, 2))

In [0]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


Create data with user, item, and target field

In [0]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.23 minutes


In [0]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [0]:
def create_data_dummy(data):
  data_dummy=data.copy()
  data_dummy['purchase_dummy']=1
  return data_dummy

In [0]:
data_dummy = create_data_dummy(data)

Normalize item values across users

To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
(df_matrix.shape)

(24429, 300)

In [0]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.142857,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [0]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [0]:

# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [0]:
train_data

customerId,productId,purchase_count
12191,44,1
11279,2,1
15897,11,1
10896,36,2
2268,5,9
437,7,2
28316,106,1
5042,152,1
13595,4,1
9993,161,1


In [0]:
test_data

customerId,productId,purchase_count
15587,13,2
26508,299,1
3893,215,1
2431,221,1
21965,257,1
22422,31,2
6348,244,1
13060,211,1
20372,73,1
8802,166,1


In [0]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [0]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

Baseline Model

Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.


Using a Popularity model as a baseline

The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
We use turicreate library for running and evaluating both baseline and collaborative filtering models below

Training data is used for model selection

In [0]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [0]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23346 users and 300 items.

Data prepared in: 0.111831s

106868 observations to process; with 300 unique items.

In [0]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 103541

recommendations finished on 2000/62483 queries. users per second: 117461

recommendations finished on 3000/62483 queries. users per second: 126444

recommendations finished on 4000/62483 queries. users per second: 133770

recommendations finished on 5000/62483 queries. users per second: 135955

...

recommendations finished on 59000/62483 queries. users per second: 130862

recommendations finished on 60000/62483 queries. users per second: 130448

recommendations finished on 61000/62483 queries. users per second: 129713

recommendations finished on 62000/62483 queries. users per second: 129801

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |     37    | 3.088803088803089  |  1   |
|     0      |    248    | 3.022222222222222  |  2   |
|     0      |     34    | 2.9655172413793105 |  3   |
|     0      |     0     | 2.951447245564893  |  4   |
|     0      |    132    | 2.9074074074074074 |  5   |
|     0      |    110    | 2.8263473053892216 |  6   |
|     0      |     27    |     2.7734375      |  7   |
|     0      |     3     | 2.764826175869121  |  8   |
|     0      |    245    | 2.6931818181818183 |  9   |
|     0      |    230    | 2.6538461538461537 |  10  |
|     1      |     37    | 3.088803088803089  |  1   |
|     1      |    248    | 3.022222222222222  |  2   |
|     1      |     34    | 2.9655172413793105 |  3   |
|     1      |     0     | 2.951447245564893  |  4   |
|     1      |    132    | 2.9074074074074074 |  5   |
|     1   

Define a model function for model selection

In [0]:

# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [0]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [0]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23332 users and 300 items.

Data prepared in: 0.1142s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 117467

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     78    |  1.0  |  1   |
|    1553    |    196    |  1.0  |  2   |
|    1553    |     61    |  1.0  |  3   |
|    1553    |     1     |  1.0  |  4   |
|    1553    |     21    |  1.0  |  5   |
|    1553    |     10    |  1.0  |  6   |
|    1553    |     65    |  1.0  |  7   |
|    1553    |    256    |  1.0  |  8   |
|    1553    |    112    |  1.0  |  9   |
|    1553    |     2     |  1.0  |  10  |
|   20400    |     78    |  1.0  |  1   |
|   20400    |    196    |  1.0  |  2   |
|   20400    |     61    |  1.0  |  3   |
|   20400    |     1     |  1.0  |  4   |
|   20400    |     21    |  1.0  |  5   |
|   20400    |     10    |  1.0  |  6   |
|   20400    |     65    |  1.0  |  7   |
|   20400    |    256    |  1.0  |  8   |
|   20400    |    112    |  1.0  |  9   |
|   20400    |     2     |  1.0  |  10  |
|   19750    |     78    |  1.0  |

In [0]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23309 users and 300 items.

Data prepared in: 0.115195s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 107782

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7816901408450704 |  1   |
|    1553    |    247    | 0.33583959899749377 |  2   |
|    1553    |    230    |  0.3270072992700724 |  3   |
|    1553    |    248    | 0.26902173913043476 |  4   |
|    1553    |    125    |  0.2565517241379307 |  5   |
|    1553    |    294    |  0.2536231884057968 |  6   |
|    1553    |    276    |  0.2385496183206107 |  7   |
|    1553    |    204    |  0.231111111111111  |  8   |
|    1553    |    155    | 0.23018867924528305 |  9   |
|    1553    |     72    | 0.22577319587628866 |  10  |
|   20400    |    226    |  0.7816901408450704 |  1   |
|   20400    |    247    | 0.33583959899749377 |  2   |
|   20400    |    230    |  0.3270072992700724 |  3   |
|   20400    |    248    | 0.26902173913043476 |  4   |
|   20400    |    125    |  0.2565517241379307 |

Notes

Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks.

In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.

If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on purchase_count variable)

In [0]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
37     3.088803
248    3.022222
34     2.965517
0      2.951447
132    2.907407
110    2.826347
27     2.773438
3      2.764826
245    2.693182
230    2.653846
82     2.626609
10     2.588415
32     2.560976
226    2.517986
129    2.485876
87     2.470339
252    2.423729
58     2.423529
18     2.373402
83     2.354839
Name: purchase_count, dtype: float64

**Collaborative Filtering**

In collaborative filtering, we would recommend items based on how similar users purchase items. For instance, if customer 1 and customer 2 bought similar items, e.g. 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.

To define similarity across users, we use the following steps:

Create a user-item matrix, where index values represent unique customer IDs and column values represent unique product IDs

Create an item-to-item similarity matrix. The idea is to calculate how similar a product is to another product. There are a number of ways of calculating this. In steps 6.1 and 6.2, we use cosine and pearson similarity measure, respectively.

To calculate similarity between products X and Y, look at all customers who have rated both these items. For example, both X and Y have been rated by customers 1 and 2.
We then create two item-vectors, v1 for item X and v2 for item Y, in the user-space of (1, 2) and then find the cosine or pearson angle/distance between these vectors. A zero angle or overlapping vectors with cosine value of 1 means total similarity (or per user, across all items, there is same rating) and an angle of 90 degree would mean cosine of 0 or no similarity.
For each customer, we then predict his likelihood to buy a product (or his purchase counts) for products that he had not bought.

For our example, we will calculate rating for user 2 in the case of item Z (target item). To calculate this we weigh the just-calculated similarity-measure between the target item and other items that customer has already bought. The weighing factor is the purchase counts given by the user to items already bought by him.
We then scale this weighted sum with the sum of similarity-measures so that the calculated rating remains within a predefined limits. Thus, the predicted rating for item Z for user 2 would be calculated using similarity measures.
While I wrote python scripts for all the process including finding similarity using python scripts (which can be found in scripts folder, we can use turicreate library for now to capture different measures like using cosine and pearson distance, and evaluate the best model.



Cosine similarity

Using purchase count

In [0]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23346 users and 300 items.

Data prepared in: 0.116939s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.05056s

recommendations finished on 1000/1000 queries. users per second: 57169

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11677020788192749  |  1   |
|    1553    |     35    | 0.07258299589157105  |  2   |
|    1553    |     1     | 0.07236067056655884  |  3   |
|    1553    |    162    | 0.06531751155853271  |  4   |
|    1553    |    105    |  0.0636749505996704  |  5   |
|    1553    |     17    | 0.061118030548095705 |  6   |
|    1553    |     21    | 0.05724612474441528  |  7   |
|    1553    |     5     | 0.044860315322875974 |  8   |
|    1553    |     85    | 0.039043545722961426 |  9   |
|    1553    |    195    | 0.03685802221298218  |  10  |
|   20400    |    280    | 0.09889405965805054  |  1   |
|   20400    |     1     | 0.041494905948638916 |  2   |
|   20400    |    122    | 0.03869813680648804  |  3   |
|   20400    |     42    | 0.03809303045272827  |  4   |
|   20400    |     6     | 0.03

Using purchase dummy

In [0]:
name='cosine'
target='purchase_dummy'
cos_dummy=model(train_data_dummy,name,user_id,item_id,target,users_to_recommend,n_rec,n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23332 users and 300 items.

Data prepared in: 0.109258s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.061726s

recommendations finished on 1000/1000 queries. users per second: 68998.8

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.09627453088760377  |  1   |
|    1553    |     35    | 0.08312979936599732  |  2   |
|    1553    |     1     | 0.08272480964660645  |  3   |
|    1553    |     5     |  0.069237220287323   |  4   |
|    1553    |     17    | 0.06073616743087769  |  5   |
|    1553    |     21    | 0.06010178327560425  |  6   |
|    1553    |     8     | 0.05409390926361084  |  7   |
|    1553    |     33    | 0.050050091743469236 |  8   |
|    1553    |     15    | 0.04747622013092041  |  9   |
|    1553    |     13    | 0.04429011344909668  |  10  |
|   20400    |     78    |         0.0          |  1   |
|   20400    |    196    |         0.0          |  2   |
|   20400    |     61    |         0.0          |  3   |
|   20400    |     1     |         0.0          |  4   |
|   20400    |     21    |     


Using normalized purchase count

In [0]:
name='cosine'
target='scaled_purchase_freq'
cos_norm=model(train_data_norm,name,user_id,item_id,target,users_to_recommend,n_rec,n_display)

Preparing data set.

Data has 106868 observations with 23309 users and 300 items.

Data prepared in: 0.118274s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.062337s

recommendations finished on 1000/1000 queries. users per second: 47092.1

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |     45    |          0.0          |  1   |
|    1553    |    129    |          0.0          |  2   |
|    1553    |    192    |          0.0          |  3   |
|    1553    |     76    |          0.0          |  4   |
|    1553    |     12    |          0.0          |  5   |
|    1553    |     93    |          0.0          |  6   |
|    1553    |    153    |          0.0          |  7   |
|    1553    |     23    |          0.0          |  8   |
|    1553    |     16    |          0.0          |  9   |
|    1553    |     25    |          0.0          |  10  |
|   20400    |     45    |          0.0          |  1   |
|   20400    |    129    |          0.0          |  2   |
|   20400    |    192    |          0.0          |  3   |
|   20400    |     76    |          0.0          |  4   |
|   20400    |

Pearson similarity

#Using purchase count

In [38]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23346 users and 300 items.

Data prepared in: 0.16929s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.085782s

recommendations finished on 1000/1000 queries. users per second: 69228.1

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |     37    |  3.08790774359206  |  1   |
|    1553    |    248    | 3.0188101304901966 |  2   |
|    1553    |     34    | 2.9646888560262217 |  3   |
|    1553    |     0     | 2.9488450021414345 |  4   |
|    1553    |    132    | 2.9074074074074083 |  5   |
|    1553    |    110    | 2.819882561775024  |  6   |
|    1553    |     27    | 2.7734374999999996 |  7   |
|    1553    |     3     | 2.7648261758691226 |  8   |
|    1553    |    245    | 2.693181818181817  |  9   |
|    1553    |    230    | 2.653023383250604  |  10  |
|   20400    |     37    | 3.0872690440605046 |  1   |
|   20400    |    248    | 3.0222222222222217 |  2   |
|   20400    |     34    |  2.96551724137931  |  3   |
|   20400    |     0     | 2.9514472455648963 |  4   |
|   20400    |    132    | 2.9074074074074083 |  5   |
|   20400 

#using purchase dummy

In [40]:
name='pearson'
target='purchase_dummy'
pear_dummy=model(train_data_dummy,name,user_id,item_id,target,users_to_recommend,n_rec,n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23332 users and 300 items.

Data prepared in: 0.136256s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.0811s

recommendations finished on 1000/1000 queries. users per second: 78106.7

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     78    |  0.0  |  1   |
|    1553    |    196    |  0.0  |  2   |
|    1553    |     61    |  0.0  |  3   |
|    1553    |     1     |  0.0  |  4   |
|    1553    |     21    |  0.0  |  5   |
|    1553    |     10    |  0.0  |  6   |
|    1553    |     65    |  0.0  |  7   |
|    1553    |    256    |  0.0  |  8   |
|    1553    |    112    |  0.0  |  9   |
|    1553    |     2     |  0.0  |  10  |
|   20400    |     78    |  0.0  |  1   |
|   20400    |    196    |  0.0  |  2   |
|   20400    |     61    |  0.0  |  3   |
|   20400    |     1     |  0.0  |  4   |
|   20400    |     21    |  0.0  |  5   |
|   20400    |     10    |  0.0  |  6   |
|   20400    |     65    |  0.0  |  7   |
|   20400    |    256    |  0.0  |  8   |
|   20400    |    112    |  0.0  |  9   |
|   20400    |     2     |  0.0  |  10  |
|   19750    |     78    |  0.0  |

#using normalized purchase count

In [41]:
name='pearson'
target='scaled_purchase_freq'
pear_norm=model(train_data_norm,name,user_id,item_id,target,users_to_recommend,n_rec,n_display)

Preparing data set.

Data has 106868 observations with 23309 users and 300 items.

Data prepared in: 0.148151s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.080826s

recommendations finished on 1000/1000 queries. users per second: 72098.1

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.781512459398995  |  1   |
|    1553    |    247    |  0.3358395989974938 |  2   |
|    1553    |    230    |  0.3266478683609161 |  3   |
|    1553    |    248    | 0.26902173913043487 |  4   |
|    1553    |    125    |  0.2564108336588432 |  5   |
|    1553    |    294    |  0.2536231884057972 |  6   |
|    1553    |    276    | 0.23824722905650392 |  7   |
|    1553    |    204    | 0.23111111111111116 |  8   |
|    1553    |    155    | 0.23018867924528308 |  9   |
|    1553    |     72    | 0.22560928310315623 |  10  |
|   20400    |    226    |  0.7816901408450705 |  1   |
|   20400    |    247    |  0.3358395989974938 |  2   |
|   20400    |    230    |  0.3270072992700729 |  3   |
|   20400    |    248    | 0.26902173913043487 |  4   |
|   20400    |    125    | 0.25655172413793104 |

In collaborative filtering above, we used two approaches: cosine and pearson distance. 

We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.

**Model Evaluation**
For evaluating recommendation engines, we can use the concept of precision-recall.

RMSE (Root Mean Squared Errors)

Measures the error of predicted values

Lesser the RMSE value, better the recommendations

**Recall**

What percentage of products that a user buys are actually recommended?

If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6

**Precision**

Out of all the recommended items, how many the user actually liked?
If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8

Why are both recall and precision important?

Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall! 

Does this mean our model is good?

We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%! This very low precision indicates that the model is not great, despite their excellent recall.
So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

Lets compare all the models we have built based on precision-recall characteristics:

In [0]:
#create initial callable models
model_w_counts=[popularity_model,cos,pear]
model_w_dummy=[pop_dummy,cos_dummy,pear_dummy]
model_w_norm=[pop_norm,cos_norm,pear_norm]


names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [43]:
#models on purchase_count
eval_count=tc.recommender.util.compare_models(test_data,model_w_counts,model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13884 queries. users per second: 61599.1

recommendations finished on 2000/13884 queries. users per second: 66328.4

recommendations finished on 3000/13884 queries. users per second: 70773.1

recommendations finished on 4000/13884 queries. users per second: 73757.2

...

recommendations finished on 10000/13884 queries. users per second: 51584.1

recommendations finished on 11000/13884 queries. users per second: 52169.8

recommendations finished on 12000/13884 queries. users per second: 52527.7

recommendations finished on 13000/13884 queries. users per second: 51016.2


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.004465571881302209 |  0.002494935360103901 |
|   2    |  0.003097090175741862 | 0.0033232622252780857 |
|   3    |  0.003409200038413527 |  0.005297185617740804 |
|   4    |  0.006698357821953329 |  0.015088496024550231 |
|   5    |  0.005704407951598996 |  0.015898181033673478 |
|   6    | 0.0052338423124939975 |  0.01753856480556223  |
|   7    |  0.005000617360167939 |  0.019644363189450488 |
|   8    |  0.005248847594353214 |  0.023499153841993924 |
|   9    |  0.004889721181856076 |  0.02464024121903641  |
|   10   |  0.004732065687121907 |  0.026666889527470608 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0659829370932197

Per User RMSE (best)
+------------+------------------------+-----

#model on purchase dummy

In [44]:
eval_dummy=tc.recommender.util.compare_models(test_data_dummy,model_w_dummy,model_names=model_w_dummy)

PROGRESS: Evaluate model PopularityRecommender


recommendations finished on 1000/13904 queries. users per second: 51271.5

recommendations finished on 2000/13904 queries. users per second: 58967.5

recommendations finished on 3000/13904 queries. users per second: 66607.5

recommendations finished on 4000/13904 queries. users per second: 70327.2

...

recommendations finished on 10000/13904 queries. users per second: 64921.6

recommendations finished on 11000/13904 queries. users per second: 66298.2

recommendations finished on 12000/13904 queries. users per second: 67015.1

recommendations finished on 13000/13904 queries. users per second: 67812.9


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.004531070195627155 | 0.0020391712717662074 |
|   2    | 0.0057177790563866515 |  0.005367502428420155 |
|   3    |  0.005681818181818176 |  0.007887065701765634 |
|   4    |  0.005807681242807821 |  0.010992252972301038 |
|   5    |  0.005566743383199111 |  0.013103926899639521 |
|   6    |  0.005286248561565016 |  0.015073736128151296 |
|   7    |  0.005517425612362328 |  0.01913517442614019  |
|   8    | 0.0057087888377445285 |  0.023002519136048234 |
|   9    |  0.005617887738140965 |  0.025739799712534466 |
|   10   |  0.005422899884925206 |  0.027771106533829755 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |

#model on normalized purchase frequency

In [45]:
eval_norm=tc.recommender.util.compare_models(test_data_norm,model_w_norm,model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13888 queries. users per second: 54722.6

recommendations finished on 2000/13888 queries. users per second: 58847.8

recommendations finished on 3000/13888 queries. users per second: 64300.4

recommendations finished on 4000/13888 queries. users per second: 66785.8

...

recommendations finished on 10000/13888 queries. users per second: 50511.4

recommendations finished on 11000/13888 queries. users per second: 51097.9

recommendations finished on 12000/13888 queries. users per second: 51074.5

recommendations finished on 13000/13888 queries. users per second: 51458


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.002736175115207367 | 0.0013822598932045952 |
|   2    | 0.0022681451612903227 |  0.002425383795991516 |
|   3    | 0.0022801459293394738 | 0.0038630758082803048 |
|   4    | 0.0018721198156681983 | 0.0042332449536510676 |
|   5    | 0.0018433179723502259 |  0.005240109392975184 |
|   6    | 0.0018361175115207383 |  0.006300977288520488 |
|   7    |  0.001944124423963139 |  0.007537944766127373 |
|   8    | 0.0018631192396313361 |  0.008223388634534935 |
|   9    | 0.0018561187916026616 |  0.009118045892599454 |
|   10   | 0.0025489631336405643 |  0.01389892329732481  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13242663284096393

Per User RMSE (best)
+------------+----------------------+------

#Model selection

Based on RMSE
1. Popularity on purchase counts: 1.0689648918289487
2. Cosine similarity on purchase counts: 1.9079706480669176
3. Pearson similarity on purchase counts: 1.0659829370932197

4. Popularity on purchase dummy: 0.0
5. Cosine similarity on purchase dummy: 0.9697089155231121
6. Pearson similarity on purchase dummy: 1.0

7. Popularity on scaled purchase counts: 0.13272231792667055
8. Cosine similarity on scaled purchase counts: 0.15973641248117876
9. Pearson similarity on scaled purchase counts: 0.13242663284096393

Notes

Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. 

Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.

Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts.
 
However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine. Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

#Final Output
In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.

We need to first return the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [46]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.126761s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.076131s

recommendations finished on 1000/1000 queries. users per second: 70502

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.10592188835144042  |  1   |
|    1553    |     1     | 0.10348175764083863  |  2   |
|    1553    |     35    | 0.07407552003860474  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     61    | 0.060317397117614746 |  5   |
|    1553    |     15    | 0.05949603319168091  |  6   |
|    1553    |     20    | 0.05683724880218506  |  7   |
|    1553    |     29    | 0.05509240627288818  |  8   |
|    1553    |     13    | 0.05373532772064209  |  9   |
|    1553    |     11    | 0.050322222709655764 |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

In [47]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,2,0.105922,1
1,1553,1,0.103482,2
2,1553,35,0.074076,3
3,1553,33,0.066861,4
4,1553,61,0.060317,5


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

In [0]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('/content/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [56]:
df_output = create_output(pear_norm, users_to_recommend, n_rec,print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 73871.6

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|248|125|294|276|204|155|72
11,226|247|230|248|125|294|276|204|155|165
12,226|247|230|248|125|294|276|204|155|72
16,226|247|230|248|125|294|276|204|155|72
21,226|247|230|248|125|294|276|204|155|72


#Customer recommendation function

In [0]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [58]:
customer_recomendation(4)

recommendedProducts    226|247|230|248|125|294|276|204|155|72
Name: 4, dtype: object

In [70]:
customer_recomendation(16)

recommendedProducts    226|247|230|248|125|294|276|204|155|72
Name: 16, dtype: object

Summary

In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with cosine and pearson measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data.